## Model Compilation Jupyter Notebook Example

This notebook shows the example of model compilation using edgeai-benchmark.

This script uses TIDL to compile a model and output in a format that edgeai-sdk can understand.

jai_benchmark is a python package provided in edgeai-benchmark that provides several functions to assist model compilation.

In [1]:
import os
import tempfile
import argparse
import cv2
from jai_benchmark import *

In [2]:
# the cwd must be the root of the respository
if os.path.split(os.getcwd())[-1] in ('scripts', 'tutorials'):
    os.chdir('../')
#
print(os.environ['TIDL_TOOLS_PATH'])
print(os.getcwd())

/data/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark/dependencies/ti-processor-sdk-rtos/tidl_j7_08_00_00_05/tidl_tools
/data/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark


#### Create a temporary directory. 

This is were the compiled artifacts will be stored.

In [4]:
modelartifacts_tempdir = tempfile.TemporaryDirectory()
print(modelartifacts_tempdir)
modelartifacts_custom = os.path.join(modelartifacts_tempdir.name, 'modelartifacts')

<TemporaryDirectory '/tmp/tmpkcvl8g2k'>


#### Read settings from settings_import_on_pc.yaml

Modify the settings as necessary in the constructor of settings.ConfigSettings()

In [5]:
settings = config_settings.ConfigSettings('./settings_import_on_pc.yaml', 
                modelartifacts_path=modelartifacts_custom,
                calibration_frames=5, calibration_iteration=5, num_frames=100)

work_dir = os.path.join(settings.modelartifacts_path, f'{settings.tensor_bits}bits')
print(f'work_dir = {work_dir}')

work_dir = /tmp/tmpkcvl8g2k/modelartifacts/8bits


#### Create Dataset Reader classes

Change the dataset paths according to your dataset location

In [6]:
dataset_calib_cfg = dict(
    path=f'{settings.datasets_path}/imagenet/val',
    split=f'{settings.datasets_path}/imagenet/val.txt',
    num_classes=1000,
    shuffle=True,
    num_frames=min(settings.calibration_frames,50000))

# dataset parameters for actual inference
dataset_val_cfg = dict(
    path=f'{settings.datasets_path}/imagenet/val',
    split=f'{settings.datasets_path}/imagenet/val.txt',
    num_classes=1000,
    shuffle=True,
    num_frames=min(settings.num_frames,50000))

calib_dataset = datasets.ImageClassification(**dataset_calib_cfg)
val_dataset = datasets.ImageClassification(**dataset_val_cfg)

#### Session runtime_options

The default runtime_options can be overriden by passing a runtime_options dict to this function

In [7]:
runtime_options_tflitert = settings.get_runtime_options(constants.SESSION_NAME_TFLITERT, is_qat=False)
print(runtime_options_tflitert)

{'tensor_bits': 8, 'accuracy_level': 1, 'debug_level': 0, 'advanced_options:high_resolution_optimization': 0, 'advanced_options:pre_batchnorm_fold': 1, 'advanced_options:calibration_frames': 5, 'advanced_options:calibration_iterations': 50, 'advanced_options:quantization_scale_type': 0, 'advanced_options:activation_clipping': 1, 'advanced_options:weight_clipping': 1, 'advanced_options:bias_calibration': 1, 'advanced_options:channel_wise_quantization': 0, 'advanced_options:output_feature_16bit_names_list': '', 'advanced_options:params_16bit_names_list': ''}


#### Create pipeline_configs

pipeline_configs is nothing but a dict with the various model configs that we want to compile

In [9]:
pipeline_configs = {
    'cl-mnv2': dict(
        task_type='classification',
        calibration_dataset=calib_dataset,
        input_dataset=val_dataset,
        preprocess=settings.get_preproc_tflite(),
        session=sessions.TFLiteRTSession(
            work_dir=work_dir, target_device=settings.target_device, runtime_options=runtime_options_tflitert,
            model_path=f'{settings.models_path}/vision/classification/imagenet1k/tf1-models/mobilenet_v2_1.0_224.tflite'),
        postprocess=settings.get_postproc_classification(),
        metric=dict(label_offset_pred=-1),
        model_info=dict(metric_reference={'accuracy_top1%':71.9})
    ),
}
print(pipeline_configs)

{'cl-mnv2': {'task_type': 'classification', 'calibration_dataset': <jai_benchmark.datasets.image_cls.ImageClassification object at 0x7febade90eb8>, 'input_dataset': <jai_benchmark.datasets.image_cls.ImageClassification object at 0x7febade90f28>, 'preprocess': <jai_benchmark.utils.transforms_utils.TransformsCompose object at 0x7febade90828>, 'session': <jai_benchmark.sessions.tflitert_session.TFLiteRTSession object at 0x7febade90e10>, 'postprocess': <jai_benchmark.utils.transforms_utils.TransformsCompose object at 0x7febade90ef0>, 'metric': {'label_offset_pred': -1}, 'model_info': {'metric_reference': {'accuracy_top1%': 71.9}}}}


#### Model Compilation

This will take a few minutes. Please be patient...

In [10]:
# run the model compliation
tools.run_accuracy(settings, work_dir, pipeline_configs)

configs to run: ['cl-mnv2_tflitert_tf1-models_mobilenet_v2_1.0_224_tflite']
number of configs: 1


TASKS                                                       |   0%|          || 0/1 [00:00<?, ?it/s]

TASKS                                                       |          |     0% 0/1| [< ]
INFO:20210805-124038: starting process on parallel_device - 0
INFO:20210805-124038: model_path - /data/files/a0393608/work/code/ti/edgeai-algo/edgeai-modelzoo/models/vision/classification/imagenet1k/tf1-models/mobilenet_v2_1.0_224.tflite
INFO:20210805-124038: model_file - /tmp/tmpkcvl8g2k/modelartifacts/8bits/cl-mnv2_tflitert_tf1-models_mobilenet_v2_1.0_224_tflite/model/mobilenet_v2_1.0_224.tflite


0it [00:00, ?it/s]


INFO:20210805-124047: running - cl-mnv2_tflitert_tf1-models_mobilenet_v2_1.0_224_tflite
INFO:20210805-124047: pipeline_config - {'task_type': 'classification', 'calibration_dataset': <jai_benchmark.datasets.image_cls.ImageClassification object at 0x7febade76fd0>, 'input_dataset': <jai_benchmark.datasets.image_cls.ImageClassification object at 0x7febade89d68>, 'preprocess': <jai_benchmark.utils.transforms_utils.TransformsCompose object at 0x7febade96d30>, 'session': <jai_benchmark.sessions.tflitert_session.TFLiteRTSession object at 0x7febade963c8>, 'postprocess': <jai_benchmark.utils.transforms_utils.TransformsCompose object at 0x7febade964e0>, 'metric': {'label_offset_pred': -1}, 'model_info': {'metric_reference': {'accuracy_top1%': 71.9}}}
INFO:20210805-124047: import  - cl-mnv2_tflitert_tf1-models_mobilenet_v2_1.0_224_tflite
INFO:20210805-124133: import completed  - cl-mnv2_tflitert_tf1-models_mobilenet_v2_1.0_224_tflite - 46 sec
INFO:20210805-124133: infer  - cl-mnv2_tflitert_tf1-m

infer : cl-mnv2_tflitert_tf1-models_mobilenet_v2_1.0_224_tfl|   0%|          || 0/100 [00:00<?, ?it/s]

infer : cl-mnv2_tflitert_tf1-models_mobilenet_v2_1.0_224_tfl|          |     0% 0/100| [< ]
INFO:20210805-124138: infer completed  - cl-mnv2_tflitert_tf1-models_mobilenet_v2_1.0_224_tflite - 5 sec

SUCCESS:20210805-124138: benchmark results - {'infer_path': 'cl-mnv2_tflitert_tf1-models_mobilenet_v2_1.0_224_tflite', 'accuracy_top1%': 74.0, 'num_subgraphs': 1, 'infer_time_core_ms': 29.988, 'infer_time_subgraph_ms': 29.979, 'ddr_transfer_mb': 0.0, 'perfsim_time_ms': 1.437, 'perfsim_ddr_transfer_mb': 4.33, 'perfsim_gmacs': 0.301}




#### Package artifacts

Package the artifacts into a .tar.gz file, keeping only the necessary files for inference.

In [11]:
out_dir = f'{work_dir}_package'
tools.package_artifacts(settings, work_dir, out_dir)

packaging artifacts to /tmp/tmpkcvl8g2k/modelartifacts/8bits_package please wait...
SUCCESS:20210805-124324: finished packaging - /tmp/tmpkcvl8g2k/modelartifacts/8bits/cl-mnv2_tflitert_tf1-models_mobilenet_v2_1.0_224_tflite


#### Download

Download the packaged .tar.gz artifact

TODO: add a download link here, that the user can click to download the packaged artifact .tar.gz file.

In [12]:
print(f'download the atricats files from the folder: {out_dir}')
print(os.listdir(out_dir))

download the atricats files from the folder: /tmp/tmpkcvl8g2k/modelartifacts/8bits_package
['cl-mnv2_tflitert_tf1-models_mobilenet_v2_1.0_224_tflite.tar.gz', 'artifacts.yaml', 'extract.sh', 'artifacts.list']


#### Cleanup

Remove the temporary folders that we created

In [13]:
# cleanup
modelartifacts_tempdir.cleanup()